# Tải thư viện

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import json
import os

# Tiền xử lý dữ liệu

In [2]:
def load_raw_data(file_path: str) -> pd.DataFrame:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} không tồn tại")
    return pd.read_csv(file_path)

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    # 1. Loại bỏ các hàng dữ liệu bị khuyết
    df = df.dropna()
    
    # 2. Loại những giá trị không hợp lệ
    df = df[(df['x'] > 0) & (df['y'] > 0) & (df['z'] > 0)]

    # 3. Chuẩn hóa các giá trị rời rạc
    for col in ['cut', 'color', 'clarity']:
        df[col] = df[col].astype(str).str.strip().str.upper()

    # 4. Loại bỏ dữ liệu trùng lặp
    df = df.drop_duplicates()

    # 5. Loại bỏ giá trị ngoại lai ở các biến liên tục sử dụng IQR
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

def save_processed_data(df: pd.DataFrame, file_path: str):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    df.to_csv(file_path, index=False)

def preprocess_pipeline(raw_path: str, processed_path: str) -> pd.DataFrame:
    df = load_raw_data(raw_path)
    df = clean_data(df)
    print("Kích thước dữ liệu sau xử lý:", df.shape, flush = True)
    save_processed_data(df, processed_path)
    return df

In [3]:
data = preprocess_pipeline(
    "../data/raw/diamonds.csv",
    "../data/processed/diamonds_clean.csv"
)

Kích thước dữ liệu sau xử lý: (46530, 11)


# Huấn luyện mô hình

## Chuẩn bị dữ liệu

In [4]:
# Các đặc trưng
features = [
    'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z'
]

# Các đặc trưng rời rạc
categorical_cols = ['cut', 'color', 'clarity']

# Các đặc trưng liên tục
numerical_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']

In [5]:
# Mã hóa các biến rời rạc

cut_order = ['FAIR', 'GOOD', 'VERY GOOD', 'PREMIUM', 'IDEAL']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

data['cut'] = data['cut'].map({v: i for i, v in enumerate(cut_order, 1)})
data['color'] = data['color'].map({v: i for i, v in enumerate(color_order, 1)})
data['clarity'] = data['clarity'].map({v: i for i, v in enumerate(clarity_order, 1)})

In [6]:
# Thực hiện các phép tính toán cần thiết trên các biến
data['log_carat'] = np.log(data['carat'])
data['volume'] = data['x'] * data['y'] * data['z']
data['x*y/z'] = data['x'] * data['y'] / data['z']
data['log_price'] = np.log(data['price'])
data['carat_cut'] = data['carat']*data['cut']
data['carat_color'] = data['carat']*data['color']
data['carat_clarity'] = data['carat']*data['clarity']

In [7]:
# Dữ liệu được chuẩn bị
data.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z,log_carat,volume,x*y/z,log_price,carat_cut,carat_color,carat_clarity
0,1,0.23,5,6,2,61.5,55.0,326,3.95,3.98,2.43,-1.469676,38.202030,6.469547,5.786897,1.15,1.38,0.46
1,2,0.21,4,6,3,59.8,61.0,326,3.89,3.84,2.31,-1.560648,34.505856,6.466494,5.786897,0.84,1.26,0.63
3,4,0.29,4,2,4,62.4,58.0,334,4.20,4.23,2.63,-1.237874,46.724580,6.755133,5.811141,1.16,0.58,1.16
4,5,0.31,2,1,2,63.3,58.0,335,4.34,4.35,2.75,-1.171183,51.917250,6.865091,5.814131,0.62,0.31,0.62
5,6,0.24,3,1,6,62.8,57.0,336,3.94,3.96,2.48,-1.427116,38.693952,6.291290,5.817111,0.72,0.24,1.44


In [8]:
data.to_csv("../data/processed/diamonds_final.csv", index=False)

In [9]:
data.reset_index(drop=True, inplace=True)
X = data.drop(columns=['price','log_price'])
y = data['price']

# Chia dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
os.makedirs("../data/splits", exist_ok=True)

# Lưu index train/test
with open("../data/splits/train_indices.json", "w") as f:
    json.dump(X_train.index.tolist(), f)
with open("../data/splits/test_indices.json", "w") as f:
    json.dump(X_test.index.tolist(), f)

## Định nghĩa mô hình

In [11]:
model_definitions = [
    # Minh
    {"Tên mô hình": "Model 1: carat", 
     "Đặc trưng": ['carat']},

    # My
    {"Tên mô hình": "Model 2: log(carat) + color + clarity + x*y/z", 
     "Đặc trưng": ['log_carat', 'color', 'clarity', 'x*y/z']},

    # Kiệt
    {"Tên mô hình": "Model 3: carat + x + y + z", 
     "Đặc trưng": ['carat', 'x', 'y', 'z']},

    # Thảo 
    {"Tên mô hình": "Model 4: log(carat) + clarity + carat_cut + carat_color + carat_clarity",
     "Đặc trưng": ['log_carat', 'clarity', 'carat_cut', 'carat_color', 'carat_clarity']},

    # Duy - log-linear regression - tranform carat and price using log 
    {"Tên mô hình": "Model 5: log(carat) + cut + color + clarity", 
     "Đặc trưng": ['log_carat', 'cut', 'color', 'clarity']},
]

## Tiến hành huấn luyện

In [12]:
def cross_val_metrics(X, y, model, k=5, log_target=False):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    mae_list, rmse_list = [], []
    
    for train_idx, val_idx in kf.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        
        if log_target:
            y_pred = np.expm1(y_pred)
            y_val = np.expm1(y_val)
        
        mae_list.append(mean_absolute_error(y_val, y_pred))
        rmse_list.append(np.sqrt(mean_squared_error(y_val, y_pred)))
    
    return np.mean(mae_list), np.mean(rmse_list)


In [13]:
results = []

for m in model_definitions:
    features = m["Đặc trưng"]


    # Pipeline: scale + LinearRegression
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])


    # Kiểm tra model dùng log target
    log_target = any(f in ['log_carat'] for f in features) or "Model 5" in m["Tên mô hình"]
    y_use = data['log_price'] if log_target else data['price']

    mae, rmse = cross_val_metrics(X_train[features], y_use.loc[X_train.index], pipeline, k=5, log_target=log_target)
    
    results.append({
        "Tên mô hình": m["Tên mô hình"],
        "Đặc trưng": features,
        "MAE trung bình": mae,
        "RMSE trung bình": rmse
    })

    pipeline.fit(X_train[features], y_use.loc[X_train.index])
    coef, inter = pipeline.named_steps['regressor'].coef_, pipeline.named_steps['regressor'].intercept_

    print(m['Tên mô hình'])
    print(f"Hệ số:")
    for f, c in zip(features, coef):
        print(f" - {f}: {c}")
    print(f"Hằng số: {inter}")
    print('-' * 50)

Model 1: carat
Hệ số:
 - carat: 2402.5155276668547
Hằng số: 3003.6722544594886
--------------------------------------------------
Model 2: log(carat) + color + clarity + x*y/z
Hệ số:
 - log_carat: 0.9248086776719776
 - color: 0.12716732136175377
 - clarity: 0.19568936646154028
 - x*y/z: 0.0602123644858259
Hằng số: 7.616119407410134
--------------------------------------------------
Model 3: carat + x + y + z
Hệ số:
 - carat: 3331.9920121533596
 - x: -1783.626937807013
 - y: 1844.3168482125652
 - z: -1002.6300569840076
Hằng số: 3003.672254459487
--------------------------------------------------
Model 4: log(carat) + clarity + carat_cut + carat_color + carat_clarity
Hệ số:
 - log_carat: 0.759735889198846
 - clarity: 0.14396938419743016
 - carat_cut: 0.033277202010040696
 - carat_color: 0.15647941764251141
 - carat_clarity: 0.06352975178878711
Hằng số: 7.616119407410134
--------------------------------------------------
Model 5: log(carat) + cut + color + clarity
Hệ số:
 - log_carat: 0.9

In [14]:
results_df = pd.DataFrame(results).sort_values(by='MAE trung bình')
results_df


,Tên mô hình,Đặc trưng,MAE trung bình,RMSE trung bình
4,Model 5: log(carat) + cut + color + clarity,"[log_carat, cut, color, clarity]",324.337800,564.047347
1,Model 2: log(carat) + color + clarity + x*y/z,"[log_carat, color, clarity, x*y/z]",330.878370,609.497826
3,Model 4: log(carat) + clarity + carat_cut + ca...,"[log_carat, clarity, carat_cut, carat_color, c...",345.165259,677.548059
2,Model 3: carat + x + y + z,"[carat, x, y, z]",607.217572,972.954837
0,Model 1: carat,[carat],648.379295,989.855725


In [15]:
best_model_def = results_df.iloc[0]
best_features = best_model_def['Đặc trưng']

best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

log_target = any(f in ['log_carat'] for f in best_features) or "Model 5" in best_model_def["Tên mô hình"]
y_use = data['log_price'] if log_target else data['price']

best_pipeline.fit(X_train[best_features], y_use.loc[X_train.index])

os.makedirs("../models", exist_ok=True)
joblib.dump(best_pipeline, "../models/best_model.pkl")
joblib.dump(best_pipeline, "../src/app/server/model.pkl")
print("Model tốt nhất đã lưu tại ../models/best_model.pkl")

Model tốt nhất đã lưu tại ../models/best_model.pkl
